In [8]:
import gensim
import pickle
from nltk.stem import WordNetLemmatizer
import json
import re
from nltk.tokenize import word_tokenize
from nltk.sentiment.util import mark_negation
import nltk
import os
import utils
from gensim.models import Word2Vec,FastText
from sklearn.decomposition import PCA
from matplotlib import pyplot

## Constants

In [2]:
data_dir="data/dataset/"
pretrained_vectors='data/GoogleNews-vectors-negative300.bin.gz'

## Read Input

In [3]:
x=[]
for dir,subdir,files in os.walk(data_dir):
    for file in files:
        file_path=os.path.join(dir,file)
        x+=utils.read_data(file_path)
        print(file_path)
print(len(x))

data/dataset/text_3.txt
data/dataset/text_7.txt
data/dataset/text_2.txt
data/dataset/text_11.txt
data/dataset/text_8.txt
data/dataset/text_6.txt
data/dataset/text_10.txt
data/dataset/text_4.txt
data/dataset/text_9.txt
data/dataset/text_1.txt
data/dataset/text_5.txt
data/dataset/text_13.txt
data/dataset/text_12.txt
93093


### Lemmatization

In [4]:
def lemmatize(lines):
    new_lines=[]
    lemmatizer=WordNetLemmatizer()
    counter=0
    for line in lines:
        counter+=1
        if(counter%10000==0):
            print(counter)
        new_line=[]
        for word in line:
            new_line.append(lemmatizer.lemmatize(word))
        new_lines.append(new_line)
    return new_lines

In [24]:
# lemm_x=lemmatize(x)
# utils.dump_pickle(lemm_x,"lemm_x.pkl")

## Training Word2Vec

In [64]:
model_2 = Word2Vec(size=300, min_count=1,workers=4,sg=1)
model_2.build_vocab(x)
total_examples = model_2.corpus_count

In [65]:
print(total_examples)
print(len(list(model_2.wv.vocab.keys())))
print(list(model_2.wv.vocab.keys())[0:10])

93093
27747
['the', 'lay', 'man', 'sermon', 'upon', 'late', 'storm', 'held', 'forth', 'at']


### Loading Google Vector

In [66]:
# model = gensim.models.KeyedVectors.load_word2vec_format(pretrained_vectors, binary=True)  

### Preparing new model with old model

In [9]:
# model_2.build_vocab([list(model.vocab.keys())], update=True)
print(list(model_2.wv.vocab.keys())[0:10])

['the', 'lay', 'man', 'sermon', 'upon', 'late', 'storm', 'held', 'forth', 'at']


In [10]:
print(list(model_2.wv.vocab.keys())[27747:27747+100])

['</s>', '##', 'The', 'I', '####', '$', '###', 'It', 'He', 'We', 'In', 'A', 'U.S.', 'But', 'percent', '#.#', 'This', '#-#', 'They', "'re", '%', '#,###', 'By', '#.##', '1', 'If', 'And', '##,###', 'That', 'AP', 'There', "'ve", 'For', '2', '##.#', 'Friday', 'billion', 'Tuesday', 'Monday', 'She', 'Thursday', '###,###', 'Wednesday', 'You', 'As', '3', "'m", 'program', 'Saturday', 'American', '##-##', 'State', 'pm', 'Sunday', '&', '5', '##th', 'New_York', 'financial', '4', 'When', '##.##', 'On', "'ll", 'With', 'County', '#:##', 'At', 'March', 'City', 'May', 'University', 'After', 'June', 'China', 'John', '6', 'To', 'April', '7', 'One', 'United_States', 'Inc.', 'Department', 'All', 'National', 'So', 'data', 'New', 'July', 'President', 'Company', '8', 'What', 'Iraq', 'Police', 'Obama', 'Washington', 'However', 'America']


In [61]:
model_2.intersect_word2vec_format(pretrained_vectors, binary=True, lockf=1.0)

### Train Model

In [67]:
model_2.train(x, total_examples=total_examples, epochs=5,report_delay=20)

(3156347, 4206920)

### Saving

In [68]:
model_path="sg_pret_emb300_ep5.pkl"
utils.dump_pickle(model_2,model_path)

In [54]:
model

## Fast-text

In [29]:
model2 = FastText(min_count=1,workers=4,min_n=1)
model2.build_vocab(x)

In [30]:
model2.corpus_count,model2.epochs

(93093, 5)

In [31]:
model2.train(x, total_examples=model2.corpus_count, epochs=40)

In [32]:
model_path="fsttxt_emb100_ep40.pkl"
utils.dump_pickle(model2,model_path)